In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

In [2]:
torch.__version__


'2.3.1'

In [3]:
#hypyer parameters 
batch_size = 64
block_size = 256
max_iters = 5000

eval_interval = 500
learning_rate = 3e-4
device ='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

n_embd = 384
n_head = 6 
n_layer = 6
dropout = 0.2

In [4]:
torch.manual_seed(1337)

In [5]:
with open('../input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# crate vocabulary list (sorted)
char = sorted(list(set(text)))
vocab_size = len(char)
vocab_size

65

In [7]:
stoi = {char: i for i, char in enumerate(char)}
itos = {i: char for i, char in enumerate(char)}


In [8]:
encoding = lambda word: [stoi[i] for i in word]
# use '' to concatenate str 
decoding = lambda number: ''.join(itos[i] for i in number)

In [9]:
#training ans tesing split 
data = torch.tensor(encoding(text), dtype=torch.int64)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [10]:
# Data loading 
# Batch in training and validation
def get_batch(set_name):
    data = (train_data if set_name == 'train' else  val_data)
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y

In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out 
        

In [12]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B, T, C = x.shape
#         X W + b 
        k = self.key(x)
        q = self.query(x)
        # attention score 
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5 # aiming to control it between 0 - 1 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        # weighted sum ---? weights @ value (x)
        out = wei @ v # (B, T, hs)
        return out

In [13]:
#multihead attention
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        # define a group of heads (nn.ModuleList including multiple head instance)
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        
        self.proj = nn.Linear(head_size * num_heads, n_embd) 
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # here's how we make num_heads * head_size to -->>>>> n_embd 
        out = self.dropout(self.proj(out))
        return out 

In [14]:
# FeedForward 
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)


In [15]:
#leogo multihead + feedforward  --> one complete transformer block! 
class Block(nn.Module):
    #transformer block 
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [16]:
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # unpacking then  Sequential can eat all of it!
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._ini_weights)
        
    def _ini_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    def forward(self, idx, targets=None):
        B, T = idx.shape 
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None 
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss 
        
    def generate(self, idx, max_new_tokens):
        # idx (B, T)
        for _ in range(max_new_tokens):
            # crop last block size column and whole batch 
            idx_cond = idx[:, -block_size:] # B, T --> Block_size
            # get predictions, forward
            logits, loss = self(idx_cond)
            # B, T, C 
            logits = logits[:, -1, :] # (B, C) only leselct last step
            probs = F.softmax(logits, dim=-1) #(B, C)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx 
            
            
            

In [17]:
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(f"cuda is available? {torch.cuda.is_available() == True}")

cuda is available? True


In [22]:
for iter in range(max_iters):
    print(f"this is {iter} iters, we are doing things!")
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train_loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
    # sample a batch of data 
    xb, yb = get_batch("train")
    
    #evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

this is 0 iters, we are doing things!
step 0: train_loss 4.2222, val loss 4.2305
this is 1 iters, we are doing things!
this is 2 iters, we are doing things!
this is 3 iters, we are doing things!
this is 4 iters, we are doing things!
this is 5 iters, we are doing things!
this is 6 iters, we are doing things!
this is 7 iters, we are doing things!
this is 8 iters, we are doing things!
this is 9 iters, we are doing things!
this is 10 iters, we are doing things!
this is 11 iters, we are doing things!
this is 12 iters, we are doing things!
this is 13 iters, we are doing things!
this is 14 iters, we are doing things!
this is 15 iters, we are doing things!
this is 16 iters, we are doing things!
this is 17 iters, we are doing things!
this is 18 iters, we are doing things!
this is 19 iters, we are doing things!
this is 20 iters, we are doing things!
this is 21 iters, we are doing things!
this is 22 iters, we are doing things!
this is 23 iters, we are doing things!
this is 24 iters, we are doing 

this is 207 iters, we are doing things!
this is 208 iters, we are doing things!
this is 209 iters, we are doing things!
this is 210 iters, we are doing things!
this is 211 iters, we are doing things!
this is 212 iters, we are doing things!
this is 213 iters, we are doing things!
this is 214 iters, we are doing things!
this is 215 iters, we are doing things!
this is 216 iters, we are doing things!
this is 217 iters, we are doing things!
this is 218 iters, we are doing things!
this is 219 iters, we are doing things!
this is 220 iters, we are doing things!
this is 221 iters, we are doing things!
this is 222 iters, we are doing things!
this is 223 iters, we are doing things!
this is 224 iters, we are doing things!
this is 225 iters, we are doing things!
this is 226 iters, we are doing things!
this is 227 iters, we are doing things!
this is 228 iters, we are doing things!
this is 229 iters, we are doing things!
this is 230 iters, we are doing things!
this is 231 iters, we are doing things!


this is 415 iters, we are doing things!
this is 416 iters, we are doing things!
this is 417 iters, we are doing things!
this is 418 iters, we are doing things!
this is 419 iters, we are doing things!
this is 420 iters, we are doing things!
this is 421 iters, we are doing things!
this is 422 iters, we are doing things!
this is 423 iters, we are doing things!
this is 424 iters, we are doing things!
this is 425 iters, we are doing things!
this is 426 iters, we are doing things!
this is 427 iters, we are doing things!
this is 428 iters, we are doing things!
this is 429 iters, we are doing things!
this is 430 iters, we are doing things!
this is 431 iters, we are doing things!
this is 432 iters, we are doing things!
this is 433 iters, we are doing things!
this is 434 iters, we are doing things!
this is 435 iters, we are doing things!
this is 436 iters, we are doing things!
this is 437 iters, we are doing things!
this is 438 iters, we are doing things!
this is 439 iters, we are doing things!


this is 620 iters, we are doing things!
this is 621 iters, we are doing things!
this is 622 iters, we are doing things!
this is 623 iters, we are doing things!
this is 624 iters, we are doing things!
this is 625 iters, we are doing things!
this is 626 iters, we are doing things!
this is 627 iters, we are doing things!
this is 628 iters, we are doing things!
this is 629 iters, we are doing things!
this is 630 iters, we are doing things!
this is 631 iters, we are doing things!
this is 632 iters, we are doing things!
this is 633 iters, we are doing things!
this is 634 iters, we are doing things!
this is 635 iters, we are doing things!
this is 636 iters, we are doing things!
this is 637 iters, we are doing things!
this is 638 iters, we are doing things!
this is 639 iters, we are doing things!
this is 640 iters, we are doing things!
this is 641 iters, we are doing things!
this is 642 iters, we are doing things!
this is 643 iters, we are doing things!
this is 644 iters, we are doing things!


this is 828 iters, we are doing things!
this is 829 iters, we are doing things!
this is 830 iters, we are doing things!
this is 831 iters, we are doing things!
this is 832 iters, we are doing things!
this is 833 iters, we are doing things!
this is 834 iters, we are doing things!
this is 835 iters, we are doing things!
this is 836 iters, we are doing things!
this is 837 iters, we are doing things!
this is 838 iters, we are doing things!
this is 839 iters, we are doing things!
this is 840 iters, we are doing things!
this is 841 iters, we are doing things!
this is 842 iters, we are doing things!
this is 843 iters, we are doing things!
this is 844 iters, we are doing things!
this is 845 iters, we are doing things!
this is 846 iters, we are doing things!
this is 847 iters, we are doing things!
this is 848 iters, we are doing things!
this is 849 iters, we are doing things!
this is 850 iters, we are doing things!
this is 851 iters, we are doing things!
this is 852 iters, we are doing things!


...

this is 4032 iters, we are doing things!
this is 4033 iters, we are doing things!
this is 4034 iters, we are doing things!
this is 4035 iters, we are doing things!
this is 4036 iters, we are doing things!
this is 4037 iters, we are doing things!
this is 4038 iters, we are doing things!
this is 4039 iters, we are doing things!
this is 4040 iters, we are doing things!
this is 4041 iters, we are doing things!
this is 4042 iters, we are doing things!
this is 4043 iters, we are doing things!
this is 4044 iters, we are doing things!
this is 4045 iters, we are doing things!
this is 4046 iters, we are doing things!
this is 4047 iters, we are doing things!
this is 4048 iters, we are doing things!
this is 4049 iters, we are doing things!
this is 4050 iters, we are doing things!
this is 4051 iters, we are doing things!
this is 4052 iters, we are doing things!
this is 4053 iters, we are doing things!
this is 4054 iters, we are doing things!
this is 4055 iters, we are doing things!
this is 4056 ite

this is 4232 iters, we are doing things!
this is 4233 iters, we are doing things!
this is 4234 iters, we are doing things!
this is 4235 iters, we are doing things!
this is 4236 iters, we are doing things!
this is 4237 iters, we are doing things!
this is 4238 iters, we are doing things!
this is 4239 iters, we are doing things!
this is 4240 iters, we are doing things!
this is 4241 iters, we are doing things!
this is 4242 iters, we are doing things!
this is 4243 iters, we are doing things!
this is 4244 iters, we are doing things!
this is 4245 iters, we are doing things!
this is 4246 iters, we are doing things!
this is 4247 iters, we are doing things!
this is 4248 iters, we are doing things!
this is 4249 iters, we are doing things!
this is 4250 iters, we are doing things!
this is 4251 iters, we are doing things!
this is 4252 iters, we are doing things!
this is 4253 iters, we are doing things!
this is 4254 iters, we are doing things!
this is 4255 iters, we are doing things!
this is 4256 ite

this is 4432 iters, we are doing things!
this is 4433 iters, we are doing things!
this is 4434 iters, we are doing things!
this is 4435 iters, we are doing things!
this is 4436 iters, we are doing things!
this is 4437 iters, we are doing things!
this is 4438 iters, we are doing things!
this is 4439 iters, we are doing things!
this is 4440 iters, we are doing things!
this is 4441 iters, we are doing things!
this is 4442 iters, we are doing things!
this is 4443 iters, we are doing things!
this is 4444 iters, we are doing things!
this is 4445 iters, we are doing things!
this is 4446 iters, we are doing things!
this is 4447 iters, we are doing things!
this is 4448 iters, we are doing things!
this is 4449 iters, we are doing things!
this is 4450 iters, we are doing things!
this is 4451 iters, we are doing things!
this is 4452 iters, we are doing things!
this is 4453 iters, we are doing things!
this is 4454 iters, we are doing things!
this is 4455 iters, we are doing things!
this is 4456 ite

this is 4632 iters, we are doing things!
this is 4633 iters, we are doing things!
this is 4634 iters, we are doing things!
this is 4635 iters, we are doing things!
this is 4636 iters, we are doing things!
this is 4637 iters, we are doing things!
this is 4638 iters, we are doing things!
this is 4639 iters, we are doing things!
this is 4640 iters, we are doing things!
this is 4641 iters, we are doing things!
this is 4642 iters, we are doing things!
this is 4643 iters, we are doing things!
this is 4644 iters, we are doing things!
this is 4645 iters, we are doing things!
this is 4646 iters, we are doing things!
this is 4647 iters, we are doing things!
this is 4648 iters, we are doing things!
this is 4649 iters, we are doing things!
this is 4650 iters, we are doing things!
this is 4651 iters, we are doing things!
this is 4652 iters, we are doing things!
this is 4653 iters, we are doing things!
this is 4654 iters, we are doing things!
this is 4655 iters, we are doing things!
this is 4656 ite

this is 4832 iters, we are doing things!
this is 4833 iters, we are doing things!
this is 4834 iters, we are doing things!
this is 4835 iters, we are doing things!
this is 4836 iters, we are doing things!
this is 4837 iters, we are doing things!
this is 4838 iters, we are doing things!
this is 4839 iters, we are doing things!
this is 4840 iters, we are doing things!
this is 4841 iters, we are doing things!
this is 4842 iters, we are doing things!
this is 4843 iters, we are doing things!
this is 4844 iters, we are doing things!
this is 4845 iters, we are doing things!
this is 4846 iters, we are doing things!
this is 4847 iters, we are doing things!
this is 4848 iters, we are doing things!
this is 4849 iters, we are doing things!
this is 4850 iters, we are doing things!
this is 4851 iters, we are doing things!
this is 4852 iters, we are doing things!
this is 4853 iters, we are doing things!
this is 4854 iters, we are doing things!
this is 4855 iters, we are doing things!
this is 4856 ite

In [25]:
#generate from model 
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decoding(m.generate(context, max_new_tokens=500)[0].tolist()))



All homeNs very many ancestor, but knowing more
plains what you have been so?

Seen Musician:
Why, the king does afresh.

DERBY:
Why stay had here?

Servant:
First, here dide his heart.

CORIOLANUS:
Marry, sir:
What news?

MENENIUS:
You have brought met, my lord?

ESCALUS:
This would the gods do fawn up into this garand;
And when lies the labour I have done to do itself.
My request king with his men's rottingons be found,
the coffus of his ages nor small well with that: lart thou fill.
With fond


In [ ]:
# embedding x --> B, T, C
